In [1]:
import os
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [5]:
class QueryEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, embed_size):
        super(QueryEncoder, self).__init__()
        
        # Word to Vector Embedding
        self.encoder = nn.Embedding(input_size, embed_size)
        
        # LSTMs
        
    def forward(self, x):
        pass
    
class ContextEncoder(nn.Module):
    def __init__(self):
        super(ContextEncoder, self).__init__()
        
        # CNNs
        
    def forward(self, x):
        pass

class MasterPolicy(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MasterPolicy, self).__init__()
        
        self.hidden_size = hidden_size
        
        # Define weights here
    
    def forward(self, x):
        pass

class E2E_RNMN(torch.nn.Module):
    def __init__(self):
        super(E2E_RNMN, self).__init__()
        
        self.query_encoder = QueryEncoder(1, 1, 1)
        self.context_encoder = ContextEncoder()
        self.master_policy = MasterPolicy(1, 1)
        self.modules = list()
    
    def forward(self, query, context):
        pass

In [6]:
# Set hyperparams and load dataset
lr = 1e-4
batch_size = 64

In [7]:
# Init model
model = E2E_RNMN()
#criterion = torch.optim.Adam(model.parameters())